# Linear Mixed Effects analysis
## Example for N400 Component (1 ROI)

---
Copyright 2022-23 [Aaron J Newman](https://github.com/aaronjnewman), [NeuroCognitive Imaging Lab](http://ncil.science), [Dalhousie University](https://dal.ca)

Released under the [BSD 3-clause license](https://opensource.org/licenses/BSD-3-Clause)

---

## Load necessary libraries

In [ ]:
date()

In [ ]:
library(tidyverse)
library(arrow)
library(ggplot2)
library(mgcv)
library(parallel)
library(emmeans)
library(sjPlot)
library(caret)
library(stargazer)
library(yaml)

## Set parameters for this analysis

In [ ]:
component <- 'n400' # chnage component name for s below as well!
subj_prefix <- 'sub-'

cfg <- read_yaml('../../config.json')
component_rois <- cfg$components$n400$rois
# num parallel processes for bam(). Should be less than the number of threads supported by your CPU
num_cores <- cfg$Preprocessing$n_jobs
# subjects to exclude
exclude <- cfg$exclude

options("scipen"=100, "digits"=5)  # force decimal instead of scientific notation

## File paths

In [ ]:
bids_root = '../..'
erp_path <- file.path(bids_root, 'derivatives', 'erp_measurement', component, 'data')
behav_path <- file.path(bids_root, 'derivatives', 'behavioral_demographic', 'data')
raw_path <- file.path(bids_root)
out_path <- file.path(bids_root, 'derivatives', 'erp_lme')

# create output directories and any higher-level directories they are nested in, if they don't exist
tab_path <- file.path(out_path, paste(component, sep=''), 'tables')
dir.create(file.path(tab_path), recursive = TRUE, showWarnings = FALSE)

fig_path <- file.path(out_path, paste(component, sep=''), 'figures')
dir.create(file.path(fig_path), recursive = TRUE, showWarnings = FALSE)

## Set options for plots

In [ ]:
set_theme(base = theme_sjplot(),
          title.size = 1.5,
          title.align = 'center',
          axis.title.size = 1.5,
          axis.textsize.x=1.1,
          axis.textsize.y=1.1,
          legend.title.size = 1.5,
          legend.item.size = 1.25,
          legend.item.backcol = 'white',
          legend.size = 1.5,
          panel.gridcol.x='white',
          panel.minor.gridcol='white'
         )

# Match colours with seaborn's colorblind palette
# in python: [matplotlib.colors.to_hex(val) for val in colors.values()]
sns_colors = c('#0173b2', '#de8f05', '#029e73', '#d55e00', '#cc78bc')         

---
## Load ERP Data

In [ ]:
files <- Sys.glob(file.path(erp_path,  paste(subj_prefix, '*', sep=''), paste('*_indiv_trials_trimmed_*.feather', sep='')))

df <- files %>% map_dfr(read_feather) %>%  
        mutate_if(is.character,as.factor) %>%
        select(-c('__index_level_0__')) %>% 
        filter( ROI %in% component_rois) %>% droplevels()

# Drop FF and CS
# df <- df %>% filter(Condition %in% c('PseudoWord', 'NovelWord', 'RealWord')) %>% droplevels()
# set order of conditions
# df$Condition <- with(df, factor(Condition,  levels = c('PseudoWord', 'NovelWord', 'RealWord')))  # 'FalseFont', 'ConsonantString', 
df$Condition <- with(df, factor(Condition,  levels = c('FalseFont', 'ConsonantString', 'PseudoWord', 'NovelWord', 'RealWord')))

# keep only conditions of interest for this component
# df <- df %>% filter( ROI %in% component_rois) %>% droplevels()
# summary(df)

## Read behavioural data from LDT

In [ ]:
acc <- read.csv(file.path(behav_path, 'ldt_acc_by_subj_wide.tsv'), sep='\t')
acc$participant_id <- as.factor(acc$participant_id)
acc %>% rename(acc_CNST = ConsonantString,
              acc_FFNT = FalseFont,
              acc_PSWD= PseudoWord,
              acc_NVWD = NovelWord,
              acc_RLWD = RealWord
              ) -> acc
# summary(acc)

In [ ]:
rt <- read.csv(file.path(behav_path, 'ldt_rt_trimmed_by_subj_wide.tsv'), sep='\t')
rt$participant_id <- as.factor(rt$participant_id)
rt %>% rename(rt_CNST = ConsonantString,
              rt_FFNT = FalseFont,
              rt_PSWD= PseudoWord,
              rt_NVWD = NovelWord,
              rt_RLWD = RealWord
              ) -> rt

# summary(rt)

### merge LDT

In [ ]:
ldt <- merge(acc, rt, by=c('participant_id'))
# summary(ldt)
df <- merge(df, ldt, by=c('participant_id'))

## Read demographic data

In [ ]:
demog <- read.csv(file.path(raw_path, 'participants.tsv'), sep='\t',
                col.names=c('participant_id', 'group', 'sex', 'hand', 'age'),
                ) %>% mutate_if(is.character,as.factor)  %>% select(-c('group'))
# summary(demog)

df <- merge(df, demog, by=c('participant_id'))

## Read d prime data and merge
calculated in `scripts/behavioral_demographic/5_LDT_stats.ipynb`

In [ ]:
dprime <- read.csv(file.path(behav_path, 'ldt_sdt_data.tsv'), sep='\t',
                   colClasses=c('factor', 'numeric', 'numeric', 'numeric', 'numeric', 'numeric'))

df <- merge(df, dprime, by='participant_id')

## Read standardized test data

In [ ]:
behav <- read.csv(file.path(raw_path, 'participants_std_tests.tsv'), 
                sep='\t') %>% mutate_if(is.character,as.factor)

## Merge ERP and behavioural data

In [ ]:
dat <- merge(df, behav, by='participant_id') %>%
            filter(!participant_id %in% exclude)  %>% droplevels()
summary(dat)

### Visualize distributions

In [ ]:
options(repr.plot.width=12, repr.plot.height=7)

ggplot(dat, aes(y=Amplitude, x=Condition)) + 
    geom_boxplot(aes(color=Condition)) 

## Run initial full model and visualize diagnostics

In [ ]:
# start timer
start_time <- Sys.time()

cl <- makeCluster(num_cores)  

mod_full <- bam(Amplitude ~ Condition + baseline 
                  + s(participant_id, bs="re")
                  + s(Channel, participant_id, bs="re") 
                  + s(Condition, participant_id, bs="re"),
                  data=dat, 
                  samfrac=0.1, cluster=cl, gc.level=2)

stopCluster(cl)

# print elpased time
end_time <- Sys.time()
print(end_time - start_time)

In [ ]:
gam.check(mod_full)

## GAM Modelling
### Compute the family of models we will consider, including primary variables

## Compare Random Effects structures

In [ ]:
# start timer
start_time <- Sys.time()
cl <- makeCluster(num_cores)

c1w_es <-  bam(Amplitude ~ Condition + baseline 
                 + s(Channel, participant_id, bs="re") 
                 + s(participant_id, bs="re"),
                  data=dat, 
                  method='ML',
                  samfrac=0.1, cluster=cl, gc.level=2)

c1w_cs <-  bam(Amplitude ~ Condition + baseline 
                 + s(Condition, participant_id, bs="re") 
                 + s(participant_id, bs="re"),
                  data=dat, 
                   method='ML',
                 samfrac=0.1, cluster=cl, gc.level=2)

c1w_escs <-  bam(Amplitude ~ Condition + baseline 
                 + s(Channel, participant_id, bs="re") 
                 + s(Condition, participant_id, bs="re") 
                 + s(participant_id, bs="re"),
                  data=dat, 
                  method='ML',
                  samfrac=0.1, cluster=cl, gc.level=2)

stopCluster(cl)
# print elpased time
end_time <- Sys.time()
print(end_time - start_time)

In [ ]:
AICtab <- as.data.frame(AIC(c1w_es, c1w_cs, c1w_escs
                           )) 
AICtab$deltaAIC <-AICtab$AIC - min(AICtab$AIC)
AICtab[order(AICtab$AIC),]

---
## View results of best model

The full model we fit at the start was confired as best via AIC comparisons

In [ ]:
mod <- mod_full

### Plot marginal means for main effect of Condition

In [ ]:
terms <- c('Condition')
out_stem <- paste(fig_path, '/', component, '_lme_me_', paste(terms, collapse='_X_'), '.pdf', sep='')

options(repr.plot.width=6, repr.plot.height=6)

pm <- function(mod, terms) {
    plot_model(mod, type='emm', terms=terms,
            title=paste(component, '- Main Effect of Condition'),
            dot.size=4, line.size=1.5,
            axis.lim=c(-4.4, -1)
            ) +
                theme(axis.text.x = element_text(angle = 20, hjust = .67, vjust=.67))
}      
pdf(out_stem, 6, 6)
pm(mod_full, terms)
dev.off() 
pm(mod_full, terms)


### Between-Condition Contrasts


In [ ]:
terms <- c('Condition')
out_stem <- paste(tab_path, '/', component, '_lme_', paste(terms, collapse='_X_'), '_contrasts', sep='')

Conditions.emm <- emmeans(mod_full, pairwise ~ Condition)
contr_tab <- as.data.frame(Conditions.emm$contrasts) 

eff_tab <- eff_size(Conditions.emm,
                     sigma = sigma(mod_full), 
                     edf = contr_tab$df[1])
eff_tab <- as.data.frame(eff_tab) 
# replace double-bracket Conditions with single-bracket to match above
eff_tab$contrast <- contr_tab$contrast
# eff_tab

full_tab <- contr_tab %>% 
                full_join(select(eff_tab, 'contrast', 'effect.size', 'lower.CL', 'upper.CL'),
                by=c('contrast'), 
                suffix = c(".ctr", ".eff"))

full_tab <- full_tab %>%
                mutate(across(c('p.value', 'effect.size'), round, 3)) %>% 
                mutate(across(c('estimate', 'SE', 't.ratio'), round, 2)) %>% 
                mutate(across(c('lower.CL', 'upper.CL'), round, 3)) %>% 
                rename('Contrast' = 'contrast',
                        'Estimate (µV)' = 'estimate',
                        't' = 't.ratio',
                        'p' = 'p.value',
                        'Effect Size' = 'effect.size',
                        'CL low' = 'lower.CL',
                        'CL high' = 'upper.CL'
                    ) %>% 
                    select('Contrast', 'Estimate (µV)', 'SE', 
                    'Effect Size', 'CL low', 'CL high', 't', 'p')  
full_tab

# Write table to CSV
# write.csv(full_tab, 
#           file=paste(out_stem, '.csv', sep='')
#          )

# Write table to LaTeX
tmp <- capture.output(stargazer(full_tab, 
                                summary=FALSE, rownames=FALSE, font.size="footnotesize",digits=NA, initial.zero=FALSE,
                                label=paste('tab:', component, '_lme_', paste(terms, collapse='_X_'), '_contr', sep=''),
                                title='Between-conditions contrasts for each condition, collapsed across hemispheres, from the linear mixed effects analysis of the N400 component from 300-400 ms. All p values are corrected for multiple comparisons using Tukey\'s method.'
                               ),
                      file=paste(out_stem, '.tex', sep='')
                     )

In [ ]:
Conditions.emm$contrasts

---
## Behavioural Correlations

### Fit all behavioral correlation models for AIC comparison


In [ ]:
# Refit all models using max liklihood so they can be compared:
start_time <- Sys.time()
cl <- makeCluster(num_cores)

mod_ortho_choice <- bam(Amplitude ~ (Condition * Ortho_Choice) + baseline
            + s(Channel, participant_id, bs="re") 
            + s(Condition, participant_id, bs="re") 
            + s(participant_id, bs="re"),
            data=dat, 
            method='ML',
            samfrac=0.1, cluster=cl, gc.level=2)            

mod_ortho_know <- bam(Amplitude ~ (Condition * Ortho_Know) + baseline
            + s(Channel, participant_id, bs="re") 
            + s(Condition, participant_id, bs="re") 
            + s(participant_id, bs="re"),
            data=dat, 
            method='ML',
            samfrac=0.1, cluster=cl, gc.level=2)            

mod_OrthoSem_Know <- bam(Amplitude ~ (Condition * OrthoSem_Know) + baseline
            + s(Channel, participant_id, bs="re") 
            + s(Condition, participant_id, bs="re") 
            + s(participant_id, bs="re"),
            data=dat, 
            method='ML',
            samfrac=0.1, cluster=cl, gc.level=2)       
            
mod_Sem_Choice <- bam(Amplitude ~ (Condition * Sem_Choice) + baseline
            + s(Channel, participant_id, bs="re") 
            + s(Condition, participant_id, bs="re") 
            + s(participant_id, bs="re"),
            data=dat, 
            method='ML',
            samfrac=0.1, cluster=cl, gc.level=2)  

mod_acc_NVWD <- bam(Amplitude ~ (Condition * acc_NVWD) + baseline
            + s(Channel, participant_id, bs="re") 
            + s(Condition, participant_id, bs="re") 
            + s(participant_id, bs="re"),
            data=dat, 
            method='ML',
            samfrac=0.1, cluster=cl, gc.level=2)  

mod_rt_NVWD <- bam(Amplitude ~ (Condition * rt_NVWD) + baseline
            + s(Channel, participant_id, bs="re") 
            + s(Condition, participant_id, bs="re") 
            + s(participant_id, bs="re"),
            data=dat, 
            method='ML',
            samfrac=0.1, cluster=cl, gc.level=2)             

mod_TOWRE <- bam(Amplitude ~ (Condition * TOWRE_idx) + baseline
            + s(Channel, participant_id, bs="re") 
            + s(Condition, participant_id, bs="re") 
            + s(participant_id, bs="re"),
            data=dat, 
            method='ML',
            samfrac=0.1, cluster=cl, gc.level=2)  

mod_PsgComp <- bam(Amplitude ~ (Condition * PsgComp_percentile) + baseline
            + s(Channel, participant_id, bs="re") 
            + s(Condition, participant_id, bs="re") 
            + s(participant_id, bs="re"),
            data=dat, 
            method='ML',
            samfrac=0.1, cluster=cl, gc.level=2)     

mod_WordID <- bam(Amplitude ~ (Condition * WordID_raw) + baseline
            + s(Channel, participant_id, bs="re") 
            + s(Condition, participant_id, bs="re") 
            + s(participant_id, bs="re"),
            data=dat, 
            method='ML',
            samfrac=0.1, cluster=cl, gc.level=2)            

mod_PPVT <- bam(Amplitude ~ (Condition * PPVT_raw) + baseline
            + s(Channel, participant_id, bs="re") 
            + s(Condition, participant_id, bs="re") 
            + s(participant_id, bs="re"),
            data=dat, 
            method='ML',
            samfrac=0.1, cluster=cl, gc.level=2)             

stopCluster(cl)
end_time <- Sys.time()
print(end_time - start_time)            

`AIC_ratio` is relative to the "base" model that did not include a behavioral variable 

In [ ]:
ref_mod <- 'c1w_escs'
AICtab <- as.data.frame(AIC(c1w_escs,
                        mod_ortho_choice, mod_ortho_know, 
                        mod_OrthoSem_Know, mod_Sem_Choice, 
                        mod_acc_NVWD, mod_rt_NVWD, 
                        mod_TOWRE, mod_PsgComp
                           ))

AICtab$deltaAIC <-AICtab$AIC - AICtab[ref_mod, ]$AIC
AICtab$AIC_L <- round(exp(-0.5 * AICtab$deltaAIC), 4)
AICtab$wAIC <- AICtab$AIC_L / sum(AICtab$AIC_L )
AICtab$AIC_ratio <-  1 / (AICtab[ref_mod, ]$wAIC / AICtab$wAIC)
AICtab[order(AICtab$AIC, decreasing=FALSE),] 

---
### Orthographic choice

In [ ]:
start_time <- Sys.time()
cl <- makeCluster(num_cores)

mod_ortho_choice <- bam(Amplitude ~ (Condition * Ortho_Choice) + baseline
            + s(Channel, participant_id, bs="re") 
            + s(Condition, participant_id, bs="re") 
            + s(participant_id, bs="re"),
            data=dat, 
            samfrac=0.1, cluster=cl, gc.level=2)            

stopCluster(cl)
end_time <- Sys.time()
print(end_time - start_time)            

In [ ]:
mod = mod_ortho_choice

In [ ]:
terms <- c('Ortho_Choice', 'Condition')

out_stem <- paste(fig_path, '/', component, '_lme_', paste(terms, collapse='_X_'), '.pdf', sep='')

options(repr.plot.width=10) #, repr.plot.height=6)

# pdf(out_stem)
# plot_model(mod, type='emm', terms=terms,
#            title=paste(component, '-', paste(terms, collapse=' X '), sep=' '),
#            dot.size=4, line.size=1.5, dodge=.33,  colors=sns_colors,
#             ) + theme(aspect.ratio=9/16, 
#                      axis.text.x = element_text(angle = 20, hjust = .5, vjust=.67)
#                      )
# dev.off() 

plot_model(mod, type='emm', terms=terms,
           title=paste(component, '-', paste(terms, collapse=' X '), sep=' '),
           dot.size=4, line.size=1.5, dodge=.33,  colors=sns_colors, 
            ) #+ theme(aspect.ratio=9/16)

In [ ]:
contr_tab <- test(emtrends(mod, ~ Condition, var='Ortho_Choice'))
ncomp = nrow(contr_tab)
contr_tab$'p (corr)' = round(p.adjust(contr_tab$'p.value', method = 'BH', n = ncomp), 4)
contr_tab <- subset(contr_tab, select=-c(p.value))
contr_tab


---
### Orthographic knowledge

In [ ]:
cl <- makeCluster(num_cores)
mod_ortho_know <- bam(Amplitude ~ (Condition * Ortho_Know) + baseline
            + s(Channel, participant_id, bs="re") 
            + s(Condition, participant_id, bs="re") 
            + s(participant_id, bs="re"),
            data=dat, 
            samfrac=0.1, cluster=cl, gc.level=2)     
stopCluster(cl)            

In [ ]:
mod <- mod_ortho_know

In [ ]:
terms <- c('Ortho_Know', 'Condition')

out_stem <- paste(fig_path, '/', component, '_lme_', paste(terms, collapse='_X_'), '.pdf', sep='')

options(repr.plot.width=10) #, repr.plot.height=6)

# pdf(out_stem)
# plot_model(mod, type='emm', terms=terms,
#            title=paste(component, '-', paste(terms, collapse=' X '), sep=' '),
#            dot.size=4, line.size=1.5, dodge=.33,  colors=sns_colors,
#             ) + theme(aspect.ratio=9/16, 
#                      axis.text.x = element_text(angle = 20, hjust = .5, vjust=.67)
#                      )
# dev.off() 

plot_model(mod, type='emm', terms=terms,
           title=paste(component, '-', paste(terms, collapse=' X '), sep=' '),
           dot.size=4, line.size=1.5, dodge=.33,  colors=sns_colors, 
            ) #+ theme(aspect.ratio=9/16)

In [ ]:
contr_tab <- test(emtrends(mod, ~ Condition, var='Ortho_Know'))
ncomp = nrow(contr_tab)
contr_tab$'p (corr)' = round(p.adjust(contr_tab$'p.value', method = 'BH', n = ncomp), 4)
contr_tab <- subset(contr_tab, select=-c(p.value))
contr_tab

---
### OrthoSem_Know

In [ ]:
start_time <- Sys.time()
cl <- makeCluster(num_cores)

mod_OrthoSem_Know <- bam(Amplitude ~ (Condition * OrthoSem_Know) + baseline
            + s(Channel, participant_id, bs="re") 
            + s(Condition, participant_id, bs="re") 
            + s(participant_id, bs="re"),
            data=dat, 
            samfrac=0.1, cluster=cl, gc.level=2)            

stopCluster(cl)
end_time <- Sys.time()
print(end_time - start_time)            

In [ ]:
mod = mod_OrthoSem_Know

In [ ]:
terms <- c('OrthoSem_Know', 'Condition')

out_stem <- paste(fig_path, '/', component, '_lme_', paste(terms, collapse='_X_'), '.pdf', sep='')

options(repr.plot.width=10) #, repr.plot.height=6)

# pdf(out_stem)
# plot_model(mod, type='emm', terms=terms,
#            title=paste(component, '-', paste(terms, collapse=' X '), sep=' '),
#            dot.size=4, line.size=1.5, dodge=.33,  colors=sns_colors,
#             ) + theme(aspect.ratio=9/16, 
#                      axis.text.x = element_text(angle = 20, hjust = .5, vjust=.67)
#                      )
# dev.off() 

plot_model(mod, type='emm', terms=terms,
           title=paste(component, '-', paste(terms, collapse=' X '), sep=' '),
           dot.size=4, line.size=1.5, dodge=.33,  colors=sns_colors, 
            ) #+ theme(aspect.ratio=9/16)

In [ ]:
contr_tab <- test(emtrends(mod, ~ Condition, var='OrthoSem_Know'))
ncomp = nrow(contr_tab)
contr_tab$'p (corr)' = round(p.adjust(contr_tab$'p.value', method = 'BH', n = ncomp), 4)
contr_tab <- subset(contr_tab, select=-c(p.value))
contr_tab

---
### Semantic Choice

In [ ]:
start_time <- Sys.time()
cl <- makeCluster(num_cores)

mod_Sem_Choice <- bam(Amplitude ~ (Condition * Sem_Choice) + baseline
            + s(Channel, participant_id, bs="re") 
            + s(Condition, participant_id, bs="re") 
            + s(participant_id, bs="re"),
            data=dat, 
            samfrac=0.1, cluster=cl, gc.level=2)            

stopCluster(cl)
end_time <- Sys.time()
print(end_time - start_time)            

In [ ]:
mod = mod_Sem_Choice

In [ ]:
terms <- c('Sem_Choice', 'Condition')

out_stem <- paste(fig_path, '/', component, '_lme_', paste(terms, collapse='_X_'), '.pdf', sep='')

options(repr.plot.width=10) #, repr.plot.height=6)

# pdf(out_stem)
# plot_model(mod, type='emm', terms=terms,
#            title=paste(component, '-', paste(terms, collapse=' X '), sep=' '),
#            dot.size=4, line.size=1.5, dodge=.33,  colors=sns_colors,
#             ) + theme(aspect.ratio=9/16, 
#                      axis.text.x = element_text(angle = 20, hjust = .5, vjust=.67)
#                      )
# dev.off() 

plot_model(mod, type='emm', terms=terms,
           title=paste(component, '-', paste(terms, collapse=' X '), sep=' '),
           dot.size=4, line.size=1.5, dodge=.33,  colors=sns_colors, 
            ) #+ theme(aspect.ratio=9/16)

In [ ]:
contr_tab <- test(emtrends(mod, ~ Condition, var='Sem_Choice'))
ncomp = nrow(contr_tab)
contr_tab$'p (corr)' = round(p.adjust(contr_tab$'p.value', method = 'BH', n = ncomp), 4)
contr_tab <- subset(contr_tab, select=-c(p.value))
contr_tab


---
### Novel Word Accuracy

In [ ]:
start_time <- Sys.time()
cl <- makeCluster(num_cores)

mod_acc_NVWD <- bam(Amplitude ~ (Condition * acc_NVWD) + baseline
            + s(Channel, participant_id, bs="re") 
            + s(Condition, participant_id, bs="re") 
            + s(participant_id, bs="re"),
            data=dat, 
            samfrac=0.1, cluster=cl, gc.level=2)            

stopCluster(cl)
end_time <- Sys.time()
print(end_time - start_time)            

In [ ]:
mod = mod_acc_NVWD

In [ ]:
terms <- c('acc_NVWD', 'Condition')

out_stem <- paste(fig_path, '/', component, '_lme_', paste(terms, collapse='_X_'), '.pdf', sep='')

options(repr.plot.width=10) #, repr.plot.height=6)

# pdf(out_stem)
# plot_model(mod, type='emm', terms=terms,
#            title=paste(component, '-', paste(terms, collapse=' X '), sep=' '),
#            dot.size=4, line.size=1.5, dodge=.33,  colors=sns_colors,
#             ) + theme(aspect.ratio=9/16, 
#                      axis.text.x = element_text(angle = 20, hjust = .5, vjust=.67)
#                      )
# dev.off() 

plot_model(mod, type='emm', terms=terms,
           title=paste(component, '-', paste(terms, collapse=' X '), sep=' '),
           dot.size=4, line.size=1.5, dodge=.33,  colors=sns_colors, 
            ) #+ theme(aspect.ratio=9/16)

In [ ]:
contr_tab <- test(emtrends(mod, ~ Condition, var='acc_NVWD'))
ncomp = nrow(contr_tab)
contr_tab$'p (corr)' = round(p.adjust(contr_tab$'p.value', method = 'BH', n = ncomp), 4)
contr_tab <- subset(contr_tab, select=-c(p.value))
contr_tab


---
### Novel Word RT

In [ ]:
start_time <- Sys.time()
cl <- makeCluster(num_cores)

mod_rt_NVWD <- bam(Amplitude ~ (Condition * rt_NVWD) + baseline
            + s(Channel, participant_id, bs="re") 
            + s(Condition, participant_id, bs="re") 
            + s(participant_id, bs="re"),
            data=dat, 
            samfrac=0.1, cluster=cl, gc.level=2)            

stopCluster(cl)
end_time <- Sys.time()
print(end_time - start_time)            

In [ ]:
mod = mod_rt_NVWD

In [ ]:
terms <- c('rt_NVWD', 'Condition')

out_stem <- paste(fig_path, '/', component, '_lme_', paste(terms, collapse='_X_'), '.pdf', sep='')

options(repr.plot.width=10) #, repr.plot.height=6)

pm <- function(mod, terms) {
        plot_model(mod, type='emm', terms=terms,
                title=paste(component, '-', paste(terms, collapse=' X '), sep=' '),
                dot.size=4, line.size=1.5, dodge=.33,  colors=sns_colors, 
                    ) 
}
pdf(out_stem, 12, 8.5)
pm(mod_rt_NVWD, terms)
dev.off() 
pm(mod_rt_NVWD, terms)

In [ ]:
contr_tab <- test(emtrends(mod_rt_NVWD, ~ Condition, var='rt_NVWD'))
# contr_tab

contr_tab <- contr_tab %>%
            mutate(across(c('p.value'), round, 3)) %>% 
            mutate(across(c('rt_NVWD.trend', 'SE', 't.ratio'), round, 2)) %>% 
            rename('Novel Word RT (slope)' = 'rt_NVWD.trend',
                   't' = 't.ratio',
                   'p' = 'p.value',
                  ) %>% 
            arrange(Condition)
contr_tab 
## Write table to LaTeX
out_stem <- paste(tab_path, '/', component, '_lme_', paste(terms, collapse='_X_'), sep='')
tmp <- capture.output(stargazer(contr_tab, 
                                summary=FALSE, rownames=FALSE,
                                label=paste('tab:', component, '_lme_', paste(terms, collapse='_X_'), sep=''),
                                title='Results of linear mixed effects model including accuracy on novel words in the lexical decision task. Statistical values are for the test of significance of the slope of the relationship between N170 amplitude and novel word accuracy. All p values are corrected for multiple comparisons using Tukey\'s method'
                               ),
                      file=paste(out_stem, '.tex', sep='')
                     )

---
### TOWRE

In [ ]:
start_time <- Sys.time()
cl <- makeCluster(num_cores)

mod_TOWRE <- bam(Amplitude ~ (Condition * TOWRE_idx) + baseline
            + s(Channel, participant_id, bs="re") 
            + s(Condition, participant_id, bs="re") 
            + s(participant_id, bs="re"),
            data=dat, 
            samfrac=0.1, cluster=cl, gc.level=2)            

stopCluster(cl)
end_time <- Sys.time()
print(end_time - start_time)            

In [ ]:
mod = mod_TOWRE

In [ ]:
terms <- c('TOWRE_idx', 'Condition')

out_stem <- paste(fig_path, '/', component, '_lme_', paste(terms, collapse='_X_'), '.pdf', sep='')

options(repr.plot.width=10) #, repr.plot.height=6)

# pdf(out_stem)
# plot_model(mod, type='emm', terms=terms,
#            title=paste(component, '-', paste(terms, collapse=' X '), sep=' '),
#            dot.size=4, line.size=1.5, dodge=.33,  colors=sns_colors,
#             ) + theme(aspect.ratio=9/16, 
#                      axis.text.x = element_text(angle = 20, hjust = .5, vjust=.67)
#                      )
# dev.off() 

plot_model(mod, type='emm', terms=terms,
           title=paste(component, '-', paste(terms, collapse=' X '), sep=' '),
           dot.size=4, line.size=1.5, dodge=.33,  colors=sns_colors, 
            ) #+ theme(aspect.ratio=9/16)

In [ ]:
contr_tab <- test(emtrends(mod, ~ Condition, var='TOWRE_idx'))
ncomp = nrow(contr_tab)
contr_tab$'p (corr)' = round(p.adjust(contr_tab$'p.value', method = 'BH', n = ncomp), 4)
contr_tab <- subset(contr_tab, select=-c(p.value))
contr_tab


---
### Word ID

In [ ]:
start_time <- Sys.time()
cl <- makeCluster(num_cores)

mod_WordID <- bam(Amplitude ~ (Condition * WordID_raw) + baseline
            + s(Channel, participant_id, bs="re") 
            + s(Condition, participant_id, bs="re") 
            + s(participant_id, bs="re"),
            data=dat, 
            samfrac=0.1, cluster=cl, gc.level=2)            

stopCluster(cl)
end_time <- Sys.time()
print(end_time - start_time)            

In [ ]:
mod = mod_WordID

In [ ]:
terms <- c('WordID_raw', 'Condition')

out_stem <- paste(fig_path, '/', component, '_lme_', paste(terms, collapse='_X_'), '.pdf', sep='')

options(repr.plot.width=10) #, repr.plot.height=6)

# pdf(out_stem)
# plot_model(mod, type='emm', terms=terms,
#            title=paste(component, '-', paste(terms, collapse=' X '), sep=' '),
#            dot.size=4, line.size=1.5, dodge=.33,  colors=sns_colors,
#             ) + theme(aspect.ratio=9/16, 
#                      axis.text.x = element_text(angle = 20, hjust = .5, vjust=.67)
#                      )
# dev.off() 

plot_model(mod, type='emm', terms=terms,
           title=paste(component, '-', paste(terms, collapse=' X '), sep=' '),
           dot.size=4, line.size=1.5, dodge=.33,  colors=sns_colors, 
            ) #+ theme(aspect.ratio=9/16)

In [ ]:
contr_tab <- test(emtrends(mod, ~ Condition, var='WordID_raw'))
ncomp = nrow(contr_tab)
contr_tab$'p (corr)' = round(p.adjust(contr_tab$'p.value', method = 'BH', n = ncomp), 4)
contr_tab <- subset(contr_tab, select=-c(p.value))
contr_tab


---
### Reading Comprehension

In [ ]:
start_time <- Sys.time()
cl <- makeCluster(num_cores)

mod_PsgComp <- bam(Amplitude ~ (Condition * PsgComp_raw) + baseline
            + s(Channel, participant_id, bs="re") 
            + s(Condition, participant_id, bs="re") 
            + s(participant_id, bs="re"),
            data=dat, 
            samfrac=0.1, cluster=cl, gc.level=2)            

stopCluster(cl)
end_time <- Sys.time()
print(end_time - start_time)            

In [ ]:
mod = mod_PsgComp

In [ ]:
terms <- c('PsgComp_raw', 'Condition')

out_stem <- paste(fig_path, '/', component, '_lme_', paste(terms, collapse='_X_'), '.pdf', sep='')

options(repr.plot.width=10) #, repr.plot.height=6)

# pdf(out_stem)
# plot_model(mod, type='emm', terms=terms,
#            title=paste(component, '-', paste(terms, collapse=' X '), sep=' '),
#            dot.size=4, line.size=1.5, dodge=.33,  colors=sns_colors,
#             ) + theme(aspect.ratio=9/16, 
#                      axis.text.x = element_text(angle = 20, hjust = .5, vjust=.67)
#                      )
# dev.off() 

plot_model(mod, type='emm', terms=terms,
           title=paste(component, '-', paste(terms, collapse=' X '), sep=' '),
           dot.size=4, line.size=1.5, dodge=.33,  colors=sns_colors, 
            ) #+ theme(aspect.ratio=9/16)

In [ ]:
contr_tab <- test(emtrends(mod, ~ Condition, var='PsgComp_raw'))
ncomp = nrow(contr_tab)
contr_tab$'p (corr)' = round(p.adjust(contr_tab$'p.value', method = 'BH', n = ncomp), 4)
contr_tab <- subset(contr_tab, select=-c(p.value))
contr_tab

---
### PPVT

In [ ]:
start_time <- Sys.time()
cl <- makeCluster(num_cores)

mod_PPVT <- bam(Amplitude ~ (Condition * PPVT_raw) + baseline
            + s(Channel, participant_id, bs="re") 
            + s(Condition, participant_id, bs="re") 
            + s(participant_id, bs="re"),
            data=dat, 
            samfrac=0.1, cluster=cl, gc.level=2)            

stopCluster(cl)
end_time <- Sys.time()
print(end_time - start_time)            

In [ ]:
mod = mod_PPVT

In [ ]:
terms <- c('PPVT_raw', 'Condition')

out_stem <- paste(fig_path, '/', component, '_lme_', paste(terms, collapse='_X_'), '.pdf', sep='')

options(repr.plot.width=10) #, repr.plot.height=6)

# pdf(out_stem)
# plot_model(mod, type='emm', terms=terms,
#            title=paste(component, '-', paste(terms, collapse=' X '), sep=' '),
#            dot.size=4, line.size=1.5, dodge=.33,  colors=sns_colors,
#             ) + theme(aspect.ratio=9/16, 
#                      axis.text.x = element_text(angle = 20, hjust = .5, vjust=.67)
#                      )
# dev.off() 

plot_model(mod, type='emm', terms=terms,
           title=paste(component, '-', paste(terms, collapse=' X '), sep=' '),
           dot.size=4, line.size=1.5, dodge=.33,  colors=sns_colors, 
            ) #+ theme(aspect.ratio=9/16)

In [ ]:
contr_tab <- test(emtrends(mod, ~ Condition, var='PPVT_raw'))
ncomp = nrow(contr_tab)
contr_tab$'p (corr)' = round(p.adjust(contr_tab$'p.value', method = 'BH', n = ncomp), 4)
contr_tab <- subset(contr_tab, select=-c(p.value))
contr_tab
